In [22]:
'''
Developer: Nick Barnette

Stable

Relevant Research:
https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7878956
https://www.researchgate.net/publication/1957378_Hash_sort_A_linear_time_complexity_multiple-dimensional_sort_algorithm
'''
import random
import time
import string
sys.setrecursionlimit(1500)

'''
List

A simple linked list implementation to handle collisions in
the hash table. The list object contains the head, tail, and 
the length of the list. I keep track of the tail of the list
so that I can append items to the list in O(1) time.

Adding an item will first add to the head when the length
is zero. After that it will set the tail of the list and then
will continue to update the tail and link in objects accordingly.

The getItems function builds an array of the values on the list.
This is a helper function for when the hash table needs to build
the sorted array. This list may or may not be fully sorted.
This is due to the shortcomings of the sorting method.

The get string function will turn the list into a string.

'''
class List():
    def __init__(self):
        self.head = ListItem(0)
        self.tail = ListItem(0)
        self.length = 0
        
    def addItem(self, v):
        li = ListItem(v)
        if self.length == 0:
            self.head = li
        elif self.length == 1:
            self.tail = li
            self.head.setNext(li)
            self.tail.setPrev(self.head)
        else:
            li.prev = self.tail
            self.tail.next = li
            self.tail = li
        self.length += 1
            
    def getItems(self):
        a = []
        if self.length == 0:
            return a
        i = self.head
        while i:
            a.append(i.value)
            if i.next:
                i = i.next
            else:
                break
        return a
    
    def getString(self):
        s = ''
        l = self.head
        while l:
            s += str(l.value) + ', '
            l = l.next
        return s[:-2]
        
        

'''
ListItem

A list item contains a previous and next node reference. This will 
allow for two-way traversal through the list. The value of the list
item is also stored.

The next and previous nodes are settable using the setters.

The str function will return the string value of the list item.

'''        
class ListItem():
    def __init__(self, v):
        self.prev = None
        self.next = None
        self.value = v
        
    def setNext(self, i):
        self.next = i
        
    def setPrev(self, i):
        self.prev = i
        
    def __str__(self):
        return self.value
    
    
    
'''
Insertion Sort

This is a quick implementation of insertion sort to compare
with the hash sort. This sorting algorithm will run in O(n^2)
time. It should work with any value that can be compared with
using the normal comparison operators.

'''
def insertionSort(a):
    print("Insertion Sorting " + str(len(a)) + " items...")
    st = time.time()
    for i, f in enumerate(a):
        if i == 0:
            continue
        n = i
        while n >= 1:
            if a[n-1] > a[n]:
                tmp = a[n]
                a[n] = a[n-1]
                a[n-1] = tmp
                n-=1
            else:
                break
    print("--- Insertion Sort: %s seconds ---" % (time.time() - st))
    return a
                
    
'''
Hash Table

Sorting strings is O(n) time.
It ends up actually being nlogk(max length of string). But since the 
length of the string is going to be a relatively low number, the length
of the string will not cause the asymtotic behavior to change.


Overview:
This implementation of the hash table is what allows for the
sorting to occur. I have created my own version of a hash table
because hash tables are implemented as dictionaries in Python.
This does not allow for the sorting effect that I am able to
acheive using this implementation.

When created, the hash table creates a table (an array) of 
a specified size. This table is initialized with the value
None in all of the positions. I also initialize the length
of the table here.

The width refers to how many characters will be sorted through
when sorting string values. The larger this number becomes, the
more space that is required by the hash table. Smaller width's
will utilize less space, but will not gaurantee that the array 
will be sorted fully.

The getSize function computes the sum of values from 1 to k where
k is the number of characters to be sorted on. This is a sum because
we will need the space for any length of string from 1 to k. For
example, if k were 2, we would need 26^1 spaces to represent the 
26 one character strings as well as 26^2 to represent the 676
two character strings.

The hash function that is currently implemented is fairly rudimentary.
For integers, it will return the integer as the hashed value. For strings,
the hashed value returns the numerical value of each of the characters which
are weighted by the position that it exists in the string and by the iteration
count. This ensures that strings are sorted according to their alphabetical 
ordering. Any hash function can be used as long as it returns an integer value.

The append function will take in the value to be appended and will hash it.
Once hashed, the hashed value is checked in the hash table. If a list does not
yet exist in this location, one is created. Once created, the value is added 
to the list. If a hashed value falls outside of the bounds of the array, it
will place it in the list on the end of the hash table. For example, if the 
largest value in the hash table is n and a hashed value turns out to be n+2,
that value will be placed in the list for n. This means that ends of the list
will not be sorted, but the algorithm may be rerun on the beginning and ends
of the hash table to ensure total sorting.

The getValues function allows the program to pull the sorted values from the
hash table as an array. Since the hash table will likely be very sparse for
a small or an undiverse input, there will be many 'None' values in the table.
This will remove the 'None' values, ensure that all values are out of the 
collision lists, and relinquishes the need to hold the large amount of space
that the hash table takes up. In this scenario, the space the hash table uses
is not freed, but I can envision that it may be in the future.

The len and str functions provide methods for determining length and converting
the structure into string form respectively. These methods are used when the
built in len() and print() functions are called respectively.


Integer Drawbacks:
There are a couple fallbacks here. The first being that the
integer values that are being sorted may not have a range from 
0 to n where n is the max number of array slots. This is an issue 
that can be addressed in a couple different ways. The first being
that a larger array may be used. This seems like a waste of memory,
but is one possibility.

The next solution that will mitigate space waste is to run through 
the list and find bounds. If for instance, the lowest value that 
needs to be sorted is 1000, we can pretend that the 0 value of the 
table is where 1000 should be placed. This allows for an extra 1000
values (n+1000 values) to be represented in the table.

The next issue that I face with this implementation when sorting 
integers is the range of values that can be sorted. For example, 
if the values to be sorted range from 0-n, but there are only n/2
spots in the array (representing values 0-n/2), values in the upper
n/2 range will not be placed in the table. This will result in a 
null pointer exception because the array slot does not exist. I am
currently addressing this issue by simply appending values that appear 
outside of the range (0-n/2) to the end of the list contained in the 
n/2th table slot. This leads to a lack of sorting for the entirety of
the unsorted items. This can then be addressed using the strategy in 
the previous paragraph. You are able to take the values in table slot
n/2 and run the hash sort algorithm on that individual slot. The lower 
bound of the array can be set to n/2 and now the second iteration of the
hash sort will run on the values that were not sorted in the first run.


String Drawbacks:
Strings are sortable in O(n) using this implementation. The drawbacks
are that for large enough unsorted data samples, there is a low granularity
for the sorted values. By this, I mean that there is a tradeoff between
table size and how many characters can be sorted. For example, to sort strings
based on the first four characters, you must have an array of 26^4 size.

To address this issue, a similar strategy can be applied as in the integer
cases. After the first run through, strings should be sorted roughly based 
on the  first k characters. Once the strings are sorted in this degree, you 
may then run the hash sort algorithm on buckets of n/2 size. This will allow
you to increase the number of characters (the width) that are being used 
when sorting.

Another performance increase to this process is by setting the bounds of the
hash table. After the first iteration has run, we know that the first k characters
of every word have been sorted. This means that we can set the 0th position in
the table to account for the fact that we don't need to sort by those characters.

Although this approach to sorting does not sort every word properly in the first
run, it should still take O(n) time. The buckets that are used when fully sorting
are dependent on how much memory one is willing to utilize as well as the length
of the words. I have not worked out the exact math here, but I do not believe 
this will cause it to approach O(nlgn) time.


Other Drawbacks:
I have not tested this method with anything other than positive integers. I 
believe that with this strategy, it is possible to develop a hash function
for any other scenario to make this sorting method work.

Large files that cannot be read into memory may also prove to have issues.
I anticipate that for a large enough data source, the strategies provided
above will not work. I am certain that there is possibility for a workaround
using physical storage. By this I mean one can treat a portion of physical 
memory as the hash table. You are able to read a portion of the large file
into memory and then sort it and then read the whole bucket back into physical
memory. This strategy could allow for a much larger hash table as physical 
memory is much larger and removes space constraints. This will allow for 
the large file to be sorted.


Updates:
Hash sort sorts values once into a hash table and then when the user
calls getValues() the lists will be sorted using insertion sort. According
to bucket sort, this should maintain O(n) for the average time complexity.
For integers, values will potentially overwhelmingly fall outside of the 
hash table range (0 - 26^6), so there may be many values in the 26^6-1 table
slot. This may be addressed by running through the values and computing
a range and then using a range to find bounds for the hash table. By this,
I mean that you can offset the array to be able to encompass more values in
the table, so the table isn't overwhelmingly skewed to one side.

'''
class HashTable():
    def __init__(self, lower=0):
        self.table = [None] * self.getSize(6)
        self.length = 0
        self.width = 5 # (width-1)
        self.lower_bound = lower
        self.upper_bound = lower + self.getSize(6)
        self.offset = self.getOffset(lower)
        
    def getSize(self, l):
        o = 0
        for i in range(1,l):
            o += pow(26,i)
        return o
    
    def getOffset(self, l):
        if l == 0:
            return 0
        v = 0
        i = 1
        while l > 1:
            l -= pow(26,i)
            v += 1
            i += 1
        return v+1
    
    def hash(self,v):
        if type(v) == str:
            o = 0
            for i,c in enumerate(v[self.offset:self.offset+self.width]):
                if i == self.width:
                    break
                o += (ord(c)-96)*pow(26,self.width-i-1)//(i+1)
            return o
        else:
            return v - self.lower_bound
    
    
    def append(self, i):
        v = self.hash(i)
        if v >= self.upper_bound:
            v = len(self.table)-1
        if self.table[v] == None:
            self.table[v] = List()
        self.table[v].addItem(i)
        self.length += 1
        
        
    def getValues(self):
        a = []
        for i in self.table:
            if i != None:
                tmp = []
                v = i.head
                while v:
                    tmp.append(v.value)
                    v = v.next
                self.insertionsort(tmp)
                for l in tmp:
                    a.append(l)
        return a
    
    def insertionsort(self,a):
        for i, f in enumerate(a):
            if i == 0:
                continue
            n = i
            while n >= 1:
                if a[n-1] > a[n]:
                    tmp = a[n]
                    a[n] = a[n-1]
                    a[n-1] = tmp
                    n-=1
                else:
                    break
        return a
        
    def __len__(self):
        return self.length
    
    def __str__(self):
        s = '['
        for i in self.table:
            if i != None:
                s += i.getString() + ', '
        s = s[:-2]
        s += ']'
        return s
        

'''
Hash Sort

This function is fairly simple. It really only needs to be three
lines of code, but I have added in some debug messages for testing
purposes. The sorting of values in this sorting method is handled
by the data structure, so there isn't much code required. You only
need to ensure the structure is created and that each value from
the unsorted list be appended to the hash table. Once finished
iterating, the values should be sorted.

'''
def hashSort(h):
    print("Hash Sorting " + str(len(h)) + " items...")
    st = time.time()
    a = HashTable()
    for i in h:
        a.append(i)
    print("--- Hash Sort: %s seconds ---" % (time.time() - st))
    return a


'''
Find Range

This function will find the min and the max of the array. This 
will allow for the array the hash table to be offset by a certain
number to save space and to attempt to prevent a large number of 
values on the right hand side.

'''
def findRange(h):
    mn = h[0]
    mx = h[0]
    for v in h:
        max(mx,v)
        min(mn,v)
    if mx-mn > pow(26,6):
        return (mx-mn)
    return mx-mn


'''
Verify Sort

This is a helper function that may be run on an array to
validate whether a sort has completed successfully or not.
If the values have been sorted properly it will print a 
success message. If there are some values out of order, 
it will print an error message as well as the number of
items that appear out of order.

'''
def verifySort(a):
    i = 0
    out = 'Values have been sorted properly!'
    num_err = 0
    while i < len(a)-1:
        if a[i] > a[i+1]:
            out = '[ERROR] Values have not been sorted properly!'
            num_err += 1
        i+=1
    print(out + ' [' + str(num_err) + ']')

    
def partition(a,l,h): 
    i = l-1
    pivot = a[h]
    for j in range(l,h): 
        if a[j] <= pivot: 
            i = i+1
            tmp = a[i]
            a[i] = a[j]
            a[j] = tmp
    tmp = a[i+1]
    a[i+1] = a[h]
    a[h] = tmp
    return i+1 

def quicksort(a,l,h): 
    if l < h:
        pivot = partition(a,l,h) 
        quicksort(a,l,pivot-1) 
        quicksort(a,pivot+1,h) 

NameError: name 'sys' is not defined

In [8]:
# Create a random array
total = 1000000

# It appears that when sorting using anything less than 
# five, words do not always end up in sorted order. This 
# may be a bug. It seems to change depending on how many
# values are being sorted.

# Max length represents the number of characters that the
# program will sort on. In this example, I have created two
# arrays. 'a' corresponds to an array with 'total' number 
# string entries of length 'width.' 'b' corresponds to an 
# array with 'total' number of integer entries.
max_length = 6
width = 4
a = []
b = []
print('Generating ' + str(total) + ' items...')
for i in range(0,total):
    s = ''
    for j in range(0,width):
        s += random.choice(string.ascii_letters)
    a.append(s.lower())
    n = random.randint(0, total-1)
    b.append(n)
print('Finished generating items.')

# The string array is sorted and the sorted hash table
# is returned. We validate that the hash table contains
# the correct amount of entries. Then run the verifySort
# function to ensure that the array has been properly
# sorted.
l = hashSort(a)
print(str(len(a)) + ' ' + str(len(l)))
print('Verifying Hash Sort (Words)...')
verifySort(l.getValues())


print('\n\n')


# The string array is sorted and the sorted hash table
# is returned. We validate that the hash table contains
# the correct amount of entries. Then run the verifySort
# function to ensure that the array has been properly
# sorted.
l = hashSort(b)
print(str(len(b)) + ' ' + str(len(l)))
print('Verifying Hash Sort (Integers)...')
verifySort(l.getValues())


print('\n\n')


q = a
print("Quick Sorting " + str(len(q)) + " items...")
st = time.time()
quicksort(q,0,len(q)-1)
print("--- Quick Sort: %s seconds ---" % (time.time() - st))
print('Verifying Quick Sort (Strings)...')
verifySort(q)


Generating 1000000 items...
Finished generating items.
Hash Sorting 1000000 items...
--- Hash Sort: 17.727391958236694 seconds ---
1000000 1000000
Verifying Hash Sort (Words)...
Values have been sorted properly! [0]



Hash Sorting 1000000 items...
--- Hash Sort: 18.456990957260132 seconds ---
1000000 1000000
Verifying Hash Sort (Integers)...
Values have been sorted properly! [0]



Quick Sorting 1000000 items...
--- Quick Sort: 7.986511945724487 seconds ---
Verifying Quick Sort (Strings)...
Values have been sorted properly! [0]


In [19]:
# Extra Hash Sorting Tests
n = [1000, 2000, 4000, 8000, 16000, 1000000, 2000000]

def intSortTests(n):
    print('Generating Items...')
    arr = []
    for i,v in enumerate(n):
        arr.append([])
        for j in range(0,v):
            arr[i].append(random.randint(0, n[i]-1))
    print('Generation Finished.')
    return arr
# g = intSortTests(n)
# avgs = []
# for a in g:
#     st = time.time()
#     l = hashSort(a)
#     avgs.append(((time.time() - st) * 1000) / len(a))
#     print(str(len(a)) + ' ' + str(len(l)))
#     print('Verifying Hash Sort (' + str(a[0].__class__.__name__) + ')...')
#     verifySort(l.getValues())
#     print('\n\n')
# print(avgs)
    
def strSortTests(n):
    print('Generating Items...')
    arr = []
    for i,v in enumerate(n):
        print('Generating ' + str(v) + ' items...')
        arr.append([])
        for j in range(0,v):
            s = ''
            for k in range(0,random.randint(1,10)):
                s += random.choice(string.ascii_letters)
            arr[i].append(s.lower())
    print('Generation Finished.')
    return arr
g = strSortTests(n)
                        
avgs = []
for a in g:
    st = time.time()
    l = hashSort(a)
    avgs.append(((time.time() - st) * 1000) / len(a))
    print(str(len(a)) + ' ' + str(len(l)))
    print('Verifying Hash Sort (' + str(a[0].__class__.__name__) + ')...')
    verifySort(l.getValues())
    print('\n\n')
    
print(avgs)

Generating Items...
Generating 1000 items...
Generating 2000 items...
Generating 4000 items...
Generating 8000 items...
Generating 16000 items...
Generating 1000000 items...
Generating 2000000 items...
Generation Finished.
Hash Sorting 1000 items...
--- Hash Sort: 0.20573902130126953 seconds ---
1000 1000
Verifying Hash Sort (str)...
Values have been sorted properly! [0]



Hash Sorting 2000 items...
--- Hash Sort: 0.25765204429626465 seconds ---
2000 2000
Verifying Hash Sort (str)...
Values have been sorted properly! [0]



Hash Sorting 4000 items...
--- Hash Sort: 0.204819917678833 seconds ---
4000 4000
Verifying Hash Sort (str)...
Values have been sorted properly! [0]



Hash Sorting 8000 items...
--- Hash Sort: 0.24103403091430664 seconds ---
8000 8000
Verifying Hash Sort (str)...
Values have been sorted properly! [0]



Hash Sorting 16000 items...
--- Hash Sort: 0.3144707679748535 seconds ---
16000 16000
Verifying Hash Sort (str)...
Values have been sorted properly! [0]



Hash So

In [23]:
avgs = []
g = strSortTests(n)
for a in g:
    print("Quick Sorting " + str(len(a)) + " items...")
    st = time.time()
    quicksort(a,0,len(a)-1)
    print("--- Hash Sort: %s seconds ---" % (time.time() - st))
    avgs.append(((time.time() - st) * 1000) / len(a))
    print('Verifying Quicksort (' + str(a[0].__class__.__name__) + ')...')
    verifySort(a)
    print('\n\n')
    
print(avgs)

Generating Items...
Generating 1000 items...
Generating 2000 items...
Generating 4000 items...
Generating 8000 items...
Generating 16000 items...
Generating 1000000 items...
Generating 2000000 items...
Generation Finished.
Quick Sorting 1000 items...
--- Hash Sort: 0.003290891647338867 seconds ---
Verifying Quicksort (str)...
Values have been sorted properly! [0]



Quick Sorting 2000 items...
--- Hash Sort: 0.08008503913879395 seconds ---
Verifying Quicksort (str)...
Values have been sorted properly! [0]



Quick Sorting 4000 items...
--- Hash Sort: 0.014373064041137695 seconds ---
Verifying Quicksort (str)...
Values have been sorted properly! [0]



Quick Sorting 8000 items...
--- Hash Sort: 0.034918785095214844 seconds ---
Verifying Quicksort (str)...
Values have been sorted properly! [0]



Quick Sorting 16000 items...
--- Hash Sort: 0.08566498756408691 seconds ---
Verifying Quicksort (str)...
Values have been sorted properly! [0]



Quick Sorting 1000000 items...


RecursionError: maximum recursion depth exceeded in comparison